<h2>The Battle of Neighborhoods in Manhattan and Downtown Toronto Boroughs</h2>

<h3>A) Importing libraries</h3>

In [1]:
# library for web scraping
!pip install bs4
from bs4 import BeautifulSoup

# html parser
!pip install html.parser

# library for working with arrays
import numpy as np

# library for data analysis
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# library to handle JSON files
import json 

# library for geocoding
!pip install geopy
from geopy.geocoders import Nominatim 

# library to handle requests
import requests 

# tranforms JSON file into pandas dataframe
from pandas.io.json import json_normalize

# plotting library
import matplotlib.cm as cm
import matplotlib.colors as colors

# k-means clustering algorithm
from sklearn.cluster import KMeans

# map rendering library
!pip install folium
import folium

print('Libraries imported.')

Libraries imported.


<h3>B) Downloading and transforming New York City data</h3>

In [2]:
# load New York data
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']

# tranform data into pandas dataframe
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

# loop through data and fill the new dataframe one row at a time
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

# check first 5 rows
neighborhoods.head()

Data downloaded!


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [3]:
# make sure that the dataframe has all 5 boroughs and 306 neighborhoods
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


<h3>C) Preparing Manhattan data</h3>

In [4]:
# create Manhattan data set
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)

# check first 5 rows
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


<h3>D) Downloading and transforming Toronto data</h3>

In [5]:
# send HTTP request to the Wikipedia URL
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# with GET request fetch the raw HTML content
html_content = requests.get(url).text

# parse the html content
soup = BeautifulSoup(html_content, "html")

# get the table having the class wikitable
toronto_table = soup.find("table", attrs={"class": "wikitable"})

# find headers
t_headers = []
for th in toronto_table.find_all("th"):
    t_headers.append(th.text.replace('\n', ' ').strip())

# find all rows from the table body
table_data = []
for tr in toronto_table.tbody.find_all("tr"):
    t_row = {}

    for td, th in zip(tr.find_all("td"), t_headers): 
        t_row[th] = td.text.replace('\n', '').strip()
        
    table_data.append(t_row)

# transform table data into dataframe
postal_codes = pd.DataFrame(data=table_data[1:], columns = t_headers)

# remove rows with not assigned borough
postal_codes = postal_codes[postal_codes['Borough'] != 'Not assigned']

# reset index
postal_codes = postal_codes.reset_index(drop=True)

# check first 5 rows
postal_codes.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
# read the csv file with geospatial data into dataframe
geospatial_data = pd.read_csv("http://cocl.us/Geospatial_data")

# add latitute and longitude from geospatial data to dataframe with postal codes
toronto_data = pd.merge(postal_codes, geospatial_data, on='Postal Code')

# remove "Postal Code" column
toronto_data=toronto_data.drop(columns='Postal Code')

# change column title
toronto_data = toronto_data.rename({'Neighbourhood': 'Neighborhood'}, axis='columns')

# check first 5 rows
toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<h3>E) Preparing Downtown Toronto data</h3> 

In [7]:
# create Downtown Toronto data set
downtown_toronto_data=toronto_data[toronto_data["Borough"]=="Downtown Toronto"].reset_index(drop=True)

# check first 5 rows
downtown_toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,Downtown Toronto,Berczy Park,43.644771,-79.373306


<h3>F) Merging Manhattan and Downtown Toronto data</h3>

In [8]:
# merge manhattan_data and downtown_toronto_data dataframes
manhattan_downtown_toronto = manhattan_data.append(downtown_toronto_data, ignore_index=True)

# check the whole data frame
manhattan_downtown_toronto

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688
5,Manhattan,Manhattanville,40.816934,-73.957385
6,Manhattan,Central Harlem,40.815976,-73.943211
7,Manhattan,East Harlem,40.792249,-73.944182
8,Manhattan,Upper East Side,40.775639,-73.960508
9,Manhattan,Yorkville,40.775930,-73.947118


<h3>G) Exploring neighborhoods in Manhattan and Downtown Toronto with Foursquare</h3>

In [9]:
# define Foursquare credentials (ID, Secret) and version
CLIENT_ID = 'URZWPVWFJKJTBUNYF1DE3N3X23IHTOKA42HYW5R4DLKEXG0H' 
CLIENT_SECRET = 'ETTOJVFVWYVGNPJATPEDAIJIAWJCPVC2OM5JDOOMCEE2APVN' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: URZWPVWFJKJTBUNYF1DE3N3X23IHTOKA42HYW5R4DLKEXG0H
CLIENT_SECRET:ETTOJVFVWYVGNPJATPEDAIJIAWJCPVC2OM5JDOOMCEE2APVN


In [10]:
# create a function that gets the nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
# run the function on each neighborhood and create a new dataframe called manhattan_downtown_toronto_venues
manhattan_downtown_toronto_venues = getNearbyVenues(names=manhattan_downtown_toronto['Neighborhood'],
                                   latitudes=manhattan_downtown_toronto['Latitude'],
                                   longitudes=manhattan_downtown_toronto['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards
Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedal

In [12]:
# check the size of the resulting dataframe
print(manhattan_downtown_toronto_venues.shape)

# check first 5 rows
manhattan_downtown_toronto_venues.head()

(4426, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


In [13]:
# check how many venues were returned for each neighborhood
manhattan_downtown_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,73,73,73,73,73,73
Berczy Park,56,56,56,56,56,56
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Carnegie Hill,86,86,86,86,86,86
Central Bay Street,63,63,63,63,63,63
Central Harlem,46,46,46,46,46,46
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Christie,17,17,17,17,17,17


In [14]:
# check the number of unique categories
print('There are {} uniques categories.'.format(len(manhattan_downtown_toronto_venues['Venue Category'].unique())))

There are 359 uniques categories.


In [15]:
# analyze each neighborhood

# one hot encoding
manhattan_downtown_toronto_onehot = pd.get_dummies(manhattan_downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_downtown_toronto_onehot['Neighborhood'] = manhattan_downtown_toronto_venues['Neighborhood'] 

# place neighborhood column as the first column
col_name='Neighborhood'
first_col = manhattan_downtown_toronto_onehot.pop(col_name)
manhattan_downtown_toronto_onehot.insert(0, col_name, first_col)

# check first 5 rows
manhattan_downtown_toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Public Art,Ramen Restau

In [16]:
# check the size of dataframe
manhattan_downtown_toronto_onehot.shape

(4426, 359)

In [17]:
# group rows by neighborhood and by taking the mean of frequency of occurrence of each category
manhattan_downtown_toronto_grouped = manhattan_downtown_toronto_onehot.groupby('Neighborhood').mean().reset_index()

# check the first 5 rows
manhattan_downtown_toronto_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Public Art,Ramen Restau

In [18]:
# confirm the new size of dataframe
manhattan_downtown_toronto_grouped.shape

(59, 359)

In [19]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in manhattan_downtown_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_downtown_toronto_grouped[manhattan_downtown_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.11
1          Hotel  0.07
2    Coffee Shop  0.05
3            Gym  0.05
4  Memorial Site  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1         Cheese Shop  0.04
2      Farmers Market  0.04
3            Beer Bar  0.04
4  Seafood Restaurant  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3       Coffee Shop  0.06
4               Bar  0.06


----Carnegie Hill----
            venue  freq
0     Coffee Shop  0.07
1            Café  0.05
2     Yoga Studio  0.03
3  Cosmetics Shop  0.03
4     Pizza Place  0.03


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.17
1   Italian Restaurant  0.05
2                 Café  0.05
3       Sandwich Place  0.05
4  Japanese Restaurant  0.03




In [20]:
# write a function that sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
# create new dataframe with top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_downtown_toronto_grouped['Neighborhood']

for ind in np.arange(manhattan_downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Gym,Coffee Shop,Memorial Site,Plaza,BBQ Joint,Sandwich Place,Shopping Mall,Playground
1,Berczy Park,Coffee Shop,Café,Cheese Shop,Restaurant,Bakery,Farmers Market,Cocktail Bar,Beer Bar,Seafood Restaurant,Tailor Shop
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Rental Car Location,Harbor / Marina,Coffee Shop,Boat or Ferry,Sculpture Garden,Boutique,Airport Gate
3,Carnegie Hill,Coffee Shop,Café,Yoga Studio,Bookstore,Gym / Fitness Center,Gym,Italian Restaurant,French Restaurant,Cosmetics Shop,Pizza Place
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Salad Place,Japanese Restaurant,Bubble Tea Shop,Falafel Restaurant,Korean Restaurant


<h3>H) Clustering Manhattan and Downtown Toronto Neighborhoods with K-means</h3>

In [22]:
# set number of clusters
kclusters = 6

# prepare dataframe for clustering
manhattan_downtown_toronto_grouped_clustering = manhattan_downtown_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_downtown_toronto_grouped_clustering)

In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_downtown_toronto_merged = manhattan_downtown_toronto

# merge data to add latitude/longitude for each neighborhood
manhattan_downtown_toronto_merged = manhattan_downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# check the whole dataframe
manhattan_downtown_toronto_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Coffee Shop,Gym,Discount Store,Sandwich Place,Yoga Studio,Pizza Place,Steakhouse,Big Box Store,Supplement Shop,Donut Shop
1,Manhattan,Chinatown,40.715618,-73.994279,5,Chinese Restaurant,Bakery,Cocktail Bar,Dessert Shop,Vietnamese Restaurant,Hotpot Restaurant,Salon / Barbershop,Optical Shop,Noodle House,Spa
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Bakery,Spanish Restaurant,Mobile Phone Shop,Chinese Restaurant,Deli / Bodega,Coffee Shop,Tapas Restaurant,New American Restaurant,Bank
3,Manhattan,Inwood,40.867684,-73.921210,0,Mexican Restaurant,Lounge,Café,Restaurant,Caribbean Restaurant,Park,Deli / Bodega,American Restaurant,Chinese Restaurant,Pizza Place
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Pizza Place,Café,Coffee Shop,Mexican Restaurant,Yoga Studio,Bakery,School,Caribbean Restaurant,Sandwich Place,Chinese Restaurant
5,Manhattan,Manhattanville,40.816934,-73.957385,0,Coffee Shop,Seafood Restaurant,Bar,Italian Restaurant,Chinese Restaurant,Mexican Restaurant,Gastropub,Sushi Restaurant,Supermarket,Lounge
6,Manhattan,Central Harlem,40.815976,-73.943211,3,African Restaurant,Chinese Restaurant,American Restaurant,Cosmetics Shop,French Restaurant,Bar,Seafood Restaurant,Tapas Restaurant,BBQ Joint,Public Art
7,Manhattan,East Harlem,40.792249,-73.944182,0,Mexican Restaurant,Bakery,Thai Restaurant,Latin American Restaurant,Spa,Deli / Bodega,Sandwich Place,Seafood Restaurant,French Restaurant,Taco Place
8,Manhattan,Upper East Side,40.775639,-73.960508,5,Italian Restaurant,Coffee Shop,Bakery,Gym / Fitness Center,Juice Bar,Yoga Studio,Hotel,French Restaurant,Exhibit,Spa
9,Manhattan,Yorkville,40.775930,-73.947118,5,Italian Restaurant,Coffee Shop,Gym,Bar,Sushi Restaurant,Japanese Restaurant,Mexican Restaurant,Pizza Place,Deli / Bodega,Wine Shop


<h3>I) Presenting clusters with Folium Map</h3>

In [24]:
# use geopy library to get the latitude and longitude values of Manhattan
address = 'Manhattan'

geolocator = Nominatim(user_agent="new_york_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [25]:
# create map of clusters in Manhattan
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_downtown_toronto_merged['Latitude'], manhattan_downtown_toronto_merged['Longitude'], manhattan_downtown_toronto_merged['Neighborhood'], manhattan_downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [26]:
# use geopy library to get the latitude and longitude values of Downtown Toronto
address = 'Downtown Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [27]:
# create map of clusters in Downtown Toronto
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_downtown_toronto_merged['Latitude'], manhattan_downtown_toronto_merged['Longitude'], manhattan_downtown_toronto_merged['Neighborhood'], manhattan_downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [28]:
# cluster 1
manhattan_downtown_toronto_merged.loc[manhattan_downtown_toronto_merged['Cluster Labels'] == 0, manhattan_downtown_toronto_merged.columns[[1] + list(range(5, manhattan_downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Bakery,Spanish Restaurant,Mobile Phone Shop,Chinese Restaurant,Deli / Bodega,Coffee Shop,Tapas Restaurant,New American Restaurant,Bank
3,Inwood,Mexican Restaurant,Lounge,Café,Restaurant,Caribbean Restaurant,Park,Deli / Bodega,American Restaurant,Chinese Restaurant,Pizza Place
4,Hamilton Heights,Pizza Place,Café,Coffee Shop,Mexican Restaurant,Yoga Studio,Bakery,School,Caribbean Restaurant,Sandwich Place,Chinese Restaurant
5,Manhattanville,Coffee Shop,Seafood Restaurant,Bar,Italian Restaurant,Chinese Restaurant,Mexican Restaurant,Gastropub,Sushi Restaurant,Supermarket,Lounge
7,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Latin American Restaurant,Spa,Deli / Bodega,Sandwich Place,Seafood Restaurant,French Restaurant,Taco Place
25,Manhattan Valley,Coffee Shop,Mexican Restaurant,Thai Restaurant,Pizza Place,Bar,Farmers Market,Bakery,Korean Restaurant,Latin American Restaurant,Furniture / Home Store
36,Tudor City,Mexican Restaurant,Café,Park,Deli / Bodega,Coffee Shop,Sushi Restaurant,Diner,Seafood Restaurant,Thai Restaurant,Dog Run
46,Christie,Grocery Store,Café,Park,Coffee Shop,Baby Store,Candy Store,Diner,Athletics & Sports,Italian Restaurant,Nightclub
51,"University of Toronto, Harbord",Café,Bookstore,Restaurant,Bar,Japanese Restaurant,Sandwich Place,Bakery,College Arts Building,Coffee Shop,Dessert Shop
52,"Kensington Market, Chinatown, Grange Park",Café,Coffee Shop,Bar,Vietnamese Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Gaming Cafe,Bakery,Arts & Crafts Store


In [29]:
# cluster 2
manhattan_downtown_toronto_merged.loc[manhattan_downtown_toronto_merged['Cluster Labels'] == 1, manhattan_downtown_toronto_merged.columns[[1] + list(range(5, manhattan_downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Coffee Shop,Gym,Discount Store,Sandwich Place,Yoga Studio,Pizza Place,Steakhouse,Big Box Store,Supplement Shop,Donut Shop
15,Midtown,Coffee Shop,Clothing Store,Bakery,Hotel,Theater,Sporting Goods Shop,Sandwich Place,Japanese Restaurant,Sushi Restaurant,Bookstore
17,Chelsea,Coffee Shop,Art Gallery,American Restaurant,Ice Cream Shop,Bakery,Hotel,Italian Restaurant,Cocktail Bar,Bar,Bookstore
29,Financial District,Coffee Shop,Pizza Place,Cocktail Bar,Steakhouse,Sandwich Place,Café,Gym / Fitness Center,Bar,Salad Place,American Restaurant
40,"Regent Park, Harbourfront",Coffee Shop,Café,Park,Bakery,Pub,Theater,Breakfast Spot,Performing Arts Venue,Farmers Market,Bank
41,"Queen's Park, Ontario Provincial Government",Coffee Shop,Yoga Studio,Portuguese Restaurant,Bar,Beer Bar,Sushi Restaurant,Smoothie Shop,Café,Sandwich Place,Chinese Restaurant
42,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Hotel,Pizza Place,Middle Eastern Restaurant,Ramen Restaurant
43,St. James Town,Coffee Shop,Café,Cocktail Bar,Restaurant,American Restaurant,Gym,Park,Lingerie Store,Clothing Store,Beer Bar
44,Berczy Park,Coffee Shop,Café,Cheese Shop,Restaurant,Bakery,Farmers Market,Cocktail Bar,Beer Bar,Seafood Restaurant,Tailor Shop
45,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Salad Place,Japanese Restaurant,Bubble Tea Shop,Falafel Restaurant,Korean Restaurant


In [30]:
# cluster 3
manhattan_downtown_toronto_merged.loc[manhattan_downtown_toronto_merged['Cluster Labels'] == 2, manhattan_downtown_toronto_merged.columns[[1] + list(range(5, manhattan_downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,Rosedale,Park,Playground,Trail,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner


In [31]:
# cluster 4
manhattan_downtown_toronto_merged.loc[manhattan_downtown_toronto_merged['Cluster Labels'] == 3, manhattan_downtown_toronto_merged.columns[[1] + list(range(5, manhattan_downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Harlem,African Restaurant,Chinese Restaurant,American Restaurant,Cosmetics Shop,French Restaurant,Bar,Seafood Restaurant,Tapas Restaurant,BBQ Joint,Public Art
11,Roosevelt Island,Park,Farmers Market,School,Deli / Bodega,Noodle House,Baseball Field,Sandwich Place,Scenic Lookout,Liquor Store,Supermarket
14,Clinton,Theater,Gym / Fitness Center,Gym,Coffee Shop,Sandwich Place,Italian Restaurant,American Restaurant,Spa,Wine Shop,Hotel
21,Tribeca,Park,Italian Restaurant,American Restaurant,Café,Wine Bar,Coffee Shop,Spa,Greek Restaurant,Bar,Poke Place
24,West Village,Italian Restaurant,New American Restaurant,American Restaurant,Park,Wine Bar,Coffee Shop,Cocktail Bar,Cosmetics Shop,Theater,Jazz Club
26,Morningside Heights,Park,Bookstore,Coffee Shop,American Restaurant,Deli / Bodega,Burger Joint,Mediterranean Restaurant,Café,Supermarket,Donut Shop
28,Battery Park City,Park,Hotel,Gym,Coffee Shop,Memorial Site,Plaza,BBQ Joint,Sandwich Place,Shopping Mall,Playground
37,Stuyvesant Town,Park,Bar,Baseball Field,Heliport,Cocktail Bar,Harbor / Marina,Coffee Shop,Pet Service,Fountain,Bistro
39,Hudson Yards,Hotel,American Restaurant,Gym / Fitness Center,Italian Restaurant,Nightclub,Restaurant,Gym,Coffee Shop,Café,Dog Run


In [32]:
# cluster 5
manhattan_downtown_toronto_merged.loc[manhattan_downtown_toronto_merged['Cluster Labels'] == 4, manhattan_downtown_toronto_merged.columns[[1] + list(range(5, manhattan_downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Rental Car Location,Harbor / Marina,Coffee Shop,Boat or Ferry,Sculpture Garden,Boutique,Airport Gate


In [33]:
# cluster 6
manhattan_downtown_toronto_merged.loc[manhattan_downtown_toronto_merged['Cluster Labels'] == 5, manhattan_downtown_toronto_merged.columns[[1] + list(range(5, manhattan_downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,Dessert Shop,Vietnamese Restaurant,Hotpot Restaurant,Salon / Barbershop,Optical Shop,Noodle House,Spa
8,Upper East Side,Italian Restaurant,Coffee Shop,Bakery,Gym / Fitness Center,Juice Bar,Yoga Studio,Hotel,French Restaurant,Exhibit,Spa
9,Yorkville,Italian Restaurant,Coffee Shop,Gym,Bar,Sushi Restaurant,Japanese Restaurant,Mexican Restaurant,Pizza Place,Deli / Bodega,Wine Shop
10,Lenox Hill,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Café,Cocktail Bar,Gym / Fitness Center,Gym,Deli / Bodega,Burger Joint
12,Upper West Side,Italian Restaurant,Coffee Shop,Bar,Wine Bar,Bakery,Indian Restaurant,Mediterranean Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Sports Bar
13,Lincoln Square,Plaza,Italian Restaurant,Café,Concert Hall,Performing Arts Venue,Theater,Indie Movie Theater,Clothing Store,Gym / Fitness Center,Wine Shop
16,Murray Hill,Sandwich Place,Hotel,Coffee Shop,Gym / Fitness Center,Italian Restaurant,American Restaurant,Japanese Restaurant,Bagel Shop,Taco Place,Bar
18,Greenwich Village,Italian Restaurant,Sushi Restaurant,Café,Clothing Store,Indian Restaurant,Dessert Shop,Chinese Restaurant,Ice Cream Shop,Gym,Vietnamese Restaurant
19,East Village,Bar,Pizza Place,Mexican Restaurant,Cocktail Bar,Coffee Shop,Ice Cream Shop,Italian Restaurant,Korean Restaurant,Vegetarian / Vegan Restaurant,Speakeasy
20,Lower East Side,Art Gallery,Chinese Restaurant,Café,Bakery,Grocery Store,Japanese Restaurant,Cocktail Bar,Coffee Shop,Ramen Restaurant,Pizza Place
